In [16]:
import pandas as pd
train_df = pd.read_table('1. 실습용자료.txt',sep='|',encoding="cp949")
test_df = pd.read_table('2. 모델개발용자료.txt',sep='|',encoding="cp949")

In [20]:
train_df.to_csv("1. 실습용자료.csv",encoding="utf-8-sig",index=False)
test_df.to_csv("2. 모델개발용자료.csv",encoding="utf-8-sig",index=False)

(100000, 7)

In [24]:
train_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매


In [27]:
test_df.head(3)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영


- text_obj, text_mthd, text_deal 3가지 텍스트 데이터를 기반으로 산업분류코드 예측하기

### 탐색적 데이터 분석(EDA)
- digit_1 : 대분류 21개
- digit_2 : 소분류 232개
- digit_3 : 세분류 495개
- '한국표준산업분류(10차)_국문.csv'의 표준산업분류 코드별 항목명 존재

실습용자료 불러오기, 열이름 수정, 열삭제

In [59]:
import pandas as pd
train_df = pd.read_csv("1. 실습용자료.csv")
train_df.columns = ["AI_id","대분류","중분류","소분류","text_obj","text_mthd","text_deal"]
#train_df.drop("AI_id",axis=1,inplace=True)

결측치 확인

In [60]:
train_df[train_df["text_obj"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
46,id_0000047,I,55,551,NaN,NaN,펜션 운영업
231,id_0000232,G,46,465,NaN,"초음파세척기, 음식물처리기 도매업",NaN
305,id_0000306,Q,87,872,NaN,재가요양서비스기관,NaN
310,id_0000311,I,55,551,NaN,NaN,펜션운영업
353,id_0000354,G,46,464,NaN,향초 도매,NaN
...,...,...,...,...,...,...,...
999765,id_0999766,G,47,472,NaN,건강분말 구매후 판매,NaN
999800,id_0999801,C,22,222,NaN,"제품입고,숏트, 철,플라스틱",NaN
999848,id_0999849,F,42,424,NaN,토탈인테리어및 유리바닥재도소매,NaN
999913,id_0999914,J,59,592,NaN,녹음시설운영업,NaN


In [61]:
train_df[train_df["text_mthd"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
21,id_0000022,R,91,911,당구,NaN,NaN
40,id_0000041,I,55,551,여관업,NaN,NaN
46,id_0000047,I,55,551,NaN,NaN,펜션 운영업
47,id_0000048,G,47,471,슈퍼마켓,NaN,NaN
58,id_0000059,G,46,467,"지붕,강판,집수리, 인테리어,태양광구조물",NaN,NaN
...,...,...,...,...,...,...,...
999975,id_0999976,C,25,259,기타임가공,NaN,NaN
999976,id_0999977,G,46,467,"고철,폐지 등",NaN,NaN
999986,id_0999987,I,56,561,음식점에서 육류 제공,NaN,NaN
999987,id_0999988,I,55,551,펜션운영업,NaN,NaN


In [62]:
train_df[train_df["text_deal"].isnull()]

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
10,id_0000011,C,10,102,멸치,"입고, 가공",NaN
21,id_0000022,R,91,911,당구,NaN,NaN
29,id_0000030,C,29,292,Resim,"재료구입, 세미앗세이",NaN
40,id_0000041,I,55,551,여관업,NaN,NaN
47,id_0000048,G,47,471,슈퍼마켓,NaN,NaN
...,...,...,...,...,...,...,...
999954,id_0999955,I,56,561,한식 음식점,NaN,NaN
999975,id_0999976,C,25,259,기타임가공,NaN,NaN
999976,id_0999977,G,46,467,"고철,폐지 등",NaN,NaN
999986,id_0999987,I,56,561,음식점에서 육류 제공,NaN,NaN


중복값 확인 : https://rfriend.tistory.com/266
- 대/중/소분류가 일치하더라도 text의 차이가 존재한다
- duplicated_df : 대/중/소분류에 중복이 존재하는 row
- 거의 모든 데이터가 대/중/소분류에 중복이 존재함
- 999064번 데이터 같은 경우는 text_obj과 text_mthd가 바꿔서 기입된 것이 아닌가 하는 의문

In [65]:
duplicated_df = train_df[train_df.duplicated(["대분류","중분류","소분류"])].sort_values(["대분류","중분류","소분류"])
duplicated_df

,AI_id,대분류,중분류,소분류,text_obj,text_mthd,text_deal
5242,id_0005243,A,1,11,벼,모판생산,농가 출하
9375,id_0009376,A,1,11,사업장에서,오이.고추를재배,오이.고추를판매
16182,id_0016183,A,1,11,벼씨,발아,묘종
20631,id_0020632,A,1,11,NaN,NaN,벼재배
20987,id_0020988,A,1,11,벼,재배하여,벼재배
...,...,...,...,...,...,...,...
998903,id_0998904,S,96,969,영업장에서,일반인을대상으로,고속화물 수하물보관소
999001,id_0999002,S,96,969,세탁소에서,고객의의뢰를받다,"세탁,드라이크리닝"
999064,id_0999065,S,96,969,사망자대상으로,화장터에서,"화장,묘지분양관리"
999794,id_0999795,S,96,969,영업장에서,일반인을대상으로,고양이카페


표준사업분류 코드별 항목명 매핑해주기
- 앞 행의 데이터로 결측치 채우기 : https://seong6496.tistory.com/84
- 특정 열의 데이터 타입 변경하기 : https://hogni.tistory.com/70

In [121]:
code_df = pd.read_csv("한국표준산업분류(10차)_국문.csv")
code_df = code_df.fillna(method="ffill")[["대분류","대분류_항목명",
                                          "중분류","중분류_항목명",
                                         "소분류","소분류_항목명",
                                         "세분류","세분류_항목명",
                                         "세세분류","세세분류_항목명"]]
code_df = code_df.astype({'중분류': 'int',
                          '소분류': 'int',
                         '세분류': 'int',
                          '세세분류': 'int'})


중복값 확인
- 세분류의 111번과 소분류의 111번은 아예 다른 표준산업분류임

In [123]:
code_df[code_df["소분류"] == 111].head(3)

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명,세분류,세분류_항목명,세세분류,세세분류_항목명
89,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11111,탁주 및 약주 제조업
90,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11112,맥아 및 맥주 제조업
91,C,제조업(10~34),11,음료 제조업,111,알코올 음료 제조업,1111,발효주 제조업,11119,기타 발효주 제조업


In [124]:
code_df[code_df["세분류"] == 111].head(3)

,대분류,대분류_항목명,중분류,중분류_항목명,소분류,소분류_항목명,세분류,세분류_항목명,세세분류,세세분류_항목명
0,A,"농업, 임업 및 어업(01~03)",1,농업,11,작물 재배업,111,곡물 및 기타 식량작물 재배업,1110,곡물 및 기타 식량작물 재배업


표준산업분류와 실습자료 연결
- 대분류/중분류/소분류_항목명 연결

In [127]:
code_df = code_df[["대분류","대분류_항목명","중분류","중분류_항목명","소분류","소분류_항목명"]]
code_df = code_df.drop_duplicates(['대분류','중분류','소분류'], keep='first')

merge_df = pd.merge(train_df,code_df,on=["대분류","중분류","소분류"],how="inner")

In [130]:
merge_df.to_csv("실습용자료_한국표준산업분류.csv",encoding="utf-8-sig",index=False)